<h4>Vector stores and retrievers</h4>
This tutorial will familiarize you with LangChain's vector store and retriever abstractions. These abstractions are designed to support retrieval of data-- from (vector) databases and other sources-- for integration with LLM workflows. They are important for applications that fetch data to be reasoned over as part of model inference, as in the case of retrieval-augmented generation.

We will cover 
- Documents
- Vector stores
- Retrievers

<h4>Documents</h4>
LangChain implements a Document abstraction, which is intended to represent a unit of text and associated metadata. It has two attributes:

- page_content: a string representing the content;
- metadata: a dict containing arbitrary metadata.
The metadata attribute can capture information about the source of the document, its relationship to other documents, and other information. Note that an individual Document object often represents a chunk of a larger document.

Let's generate some sample documents:


In [2]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]
documents[3].page_content

'Parrots are intelligent birds capable of mimicking human speech.'

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
groq_api_key=os.getenv("GROQ_API_KEY")

os.environ["HUGGINGFACEHUB_API_TOKEN"]=os.getenv("HUGGINGFACEHUB_API_TOKEN")

llm=ChatGroq(groq_api_key=groq_api_key,model="Llama3-8b-8192")
llm

c:\Users\Nitin Mishra\OneDrive\Desktop\AIML\Gen-AI\Langchain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ChatGroq(profile={}, client=<groq.resources.chat.completions.Completions object at 0x0000027A4A0E2360>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000027A4A23AA80>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [5]:
## VectorStores
from langchain_chroma import Chroma

vectorstore=Chroma.from_documents(documents,embedding=embeddings)
vectorstore


In [7]:
vectorstore.similarity_search("which pet can mimic human speech?")

[Document(id='9baace69-f493-4d39-8baf-b7dd9443fde2', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
 Document(id='8731bf4f-9f39-4670-9b2d-9147624124f4', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='c0fb5837-71e3-45f6-a10e-b59d73b72af5', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='a07ee266-cf4e-4112-8317-a9dbd0f5604b', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.')]

In [10]:
## Async query
await vectorstore.asimilarity_search("cat")

[Document(id='c0fb5837-71e3-45f6-a10e-b59d73b72af5', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='8731bf4f-9f39-4670-9b2d-9147624124f4', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='fe0bf1f1-ed09-471b-9e28-08451467409c', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(id='9baace69-f493-4d39-8baf-b7dd9443fde2', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [9]:
vectorstore.similarity_search_with_score("which pet can mimic human speech?")

[(Document(id='9baace69-f493-4d39-8baf-b7dd9443fde2', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
  0.7978457808494568),
 (Document(id='8731bf4f-9f39-4670-9b2d-9147624124f4', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  1.374259352684021),
 (Document(id='c0fb5837-71e3-45f6-a10e-b59d73b72af5', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  1.5661884546279907),
 (Document(id='a07ee266-cf4e-4112-8317-a9dbd0f5604b', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
  1.580483317375183)]

### Retrievers
LangChain VectorStore objects do not subclass Runnable, and so cannot immediately be integrated into LangChain Expression Language chains.

LangChain Retrievers are Runnables, so they implement a standard set of methods (e.g., synchronous and asynchronous invoke and batch operations) and are designed to be incorporated in LCEL chains.

We can create a simple version of this ourselves, without subclassing Retriever. If we choose what method we wish to use to retrieve documents, we can create a runnable easily. Below we will build one around the similarity_search method:

<h4>RETRIEVAL CHAIN:</h4>
- A Retrieval Chain is a process used in GenAI where the system first searches for relevant information (context) from a database or documents, and then sends that information along with the user’s question to the LLM.
- This helps the model give more accurate and context-aware answers, especially when the model does not already know the information.

<h4>WE MAKE VECTORSTORE DATABASE AND CONVERT IT INTO RETRIEVER.</h4>

<h5>WHAT DOES RETRIEVER DO AFTER THAT?:-</h5>
We don’t want to manually choose or load documents every time someone asks a question.
Instead, we want the retriever (a search mechanism) to automatically find the most relevant documents based on the user’s query.

So when a question comes in:

1. Retriever searches all available documents

2. Picks only the most useful/relevant ones

3. Sends those selected documents (context) to the LLM

4. LLM uses that context to generate a better and accurate answer

<h4>Why is this important?</h4>

- Because the LLM doesn’t always know everything on its own, and it shouldn’t read all documents every time (that would be slow and costly).
Instead, it should only receive information that matters for the current question.

<h5>Simple Analogy</h5>

Imagine you have a large library. Instead of reading all the books whenever  someone asks a question:

- The retriever acts like a librarian

- You ask the librarian your question

- The librarian quickly finds only the few books or pages that matter

- Then you read those pages to answer the question correctly.

<h5>Final Simple Statement</h5>

- The retriever helps automatically find and send only the most relevant    documents to the LLM so that the answer is accurate and context-aware for  each question.

In [ ]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever=RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(["cat","dog"])